In [24]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.utils import get_custom_objects

def cecotti_normal(shape, dtype = None, partition_info = None):
    '''
    Initializer proposed by Cecotti et al. 2011:
    https://ieeexplore.ieee.org/document/5492691
    '''
    if len(shape) == 1:
        fan_in = shape[0]
    elif len(shape) == 2:
        fan_in = shape[0]
    else:
        receptive_field_size = 1
        for dim in shape[:-2]:
            receptive_field_size *= dim
        fan_in = shape[-2] * receptive_field_size

    return K.random_normal(shape, mean = 0.0, stddev = (1.0 / fan_in))

def scaled_tanh(z):
    '''
    Scaled hyperbolic tangent activation function, as proposed
    by Lecun 1989:
    http://yann.lecun.com/exdb/publis/pdf/lecun-89.pdf

    See also Lecun et al. 1998:
    http://yann.lecun.com/exdb/publis/pdf/lecun-98b.pdf
    '''
    return 1.7159 * K.tanh((2.0 / 3.0) * z)

get_custom_objects().update({'scaled_tanh': Activation(scaled_tanh)})

def CNN1(Chans = 6, Samples = 206):
    eeg_input    = Input(shape = (Samples, Chans))

    block1       = Conv1D(10, 1, padding = 'same',
                          data_format = 'channels_last',
                          bias_initializer = cecotti_normal,
                          kernel_initializer = cecotti_normal,
                          use_bias = True)(eeg_input)
    block1       = Activation('scaled_tanh')(block1)

    block1       = Conv1D(50, 13, padding = 'same',
                          data_format = 'channels_last',
                          bias_initializer = cecotti_normal,
                          kernel_initializer = cecotti_normal,
                          use_bias = True)(block1)
    block1       = Activation('scaled_tanh')(block1)

    flatten      = Flatten(name = 'flatten')(block1)
    dense        = Dense(100, activation = 'sigmoid')(flatten)
    prediction   = Dense(2, activation = 'sigmoid')(dense)

    return Model(inputs = eeg_input, outputs = prediction, name = 'CNN1')

In [25]:
model = CNN1()


# Import Data

In [26]:
import mne
filepath = "/workspace/data/EEG/data/"

ep1 = mne.read_epochs(filepath + "epochs/A01-epo.fif", preload=True)

Reading /workspace/data/EEG/data/epochs/A01-epo.fif ...
    Found the data of interest:
        t =    -199.22 ...     800.78 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
4200 matching events found
No baseline correction applied
0 projection items activated


In [27]:
labels = ep1.events[:, -1]

In [28]:
X = ep1.get_data() * 1000
y = labels-1

In [29]:
kernels, chans, samples = 16, 8, 257

In [30]:
X = X.reshape(X.shape[0], samples, chans)

In [31]:
# take 50/20/20 percent of the data to train/validate/test
X_train      = X[0: int(0.6*len(y)),]
Y_train      = y[0: int(0.6*len(y)),]
X_validate   = X[int(0.6*len(y)) : int(0.8*len(y)) ,]
Y_validate   = y[int(0.6*len(y)) : int(0.8*len(y)) ,]
X_test       = X[int(0.8*len(y)) : len(y) ,]
Y_test       = y[int(0.8*len(y)) : len(y) ,]


In [32]:

X_train      = X_train.reshape(X_train.shape[0], samples, chans)
X_validate   = X_validate.reshape(X_validate.shape[0], samples, chans)
X_test       = X_test.reshape(X_test.shape[0], samples, chans)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (2520, 257, 8)
2520 train samples
840 test samples


In [33]:
import argparse
import sys
import numpy as np
#from CNN1 import CNN1
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
#from tensorflow import set_random_seed
from sklearn.model_selection import *
from utils import *
import tensorflow.keras.backend as K
def evaluate_subject_models(data, labels, modelpath):
    """
    Trains and evaluates CNN1 for each subject in the P300 Speller database
    using repeated stratified K-fold cross validation.
    """
    #n_sub = data.shape[0]
    #n_ex_sub = data.shape[1]
    n_samples = 257
    n_channels = 8
    aucs = np.zeros(5 * 10)
    print("Training for subject {0}: ")
    cv = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 10, random_state = 123)
    for k, (t, v) in enumerate(cv.split(data , labels )):
        X_train, y_train, X_test, y_test = data[t, :, :], labels[t], data[ v, :, :], labels[v]
        X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.2, shuffle = True, random_state = 456)
        print('Partition {0}: X_train = {1}, X_valid = {2}, X_test = {3}'.format(k, X_train.shape, X_valid.shape, X_test.shape))

        # channel-wise feature standarization
        sc = EEGChannelScaler(n_channels = n_channels)
        X_train = sc.fit_transform(X_train)
        X_valid = sc.transform(X_valid)
        X_test = sc.transform(X_test)

        model = CNN1(Chans = n_channels, Samples = n_samples)
        print(model.summary())
        model.compile(optimizer = 'adam', loss = 'mean_squared_error')

        # Early stopping setting also follows EEGNet (Lawhern et al., 2018)
        es = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 50, restore_best_weights = True)
        history = model.fit(X_train,
                            to_categorical(y_train),
                            batch_size = 256,
                            epochs = 200,
                            validation_data = (X_valid, to_categorical(y_valid)),
                            callbacks = [es])

        proba_test = model.predict(X_test)
        aucs[k] = roc_auc_score(y_test, proba_test[:, 1])
        #print('S{0}, P{1} -- AUC: {2}'.format( k, aucs[k]))
        K.clear_session()

    np.savetxt(modelpath + '/s' + '_aucs.npy', aucs)
    return aucs

In [34]:
aucs = evaluate_subject_models(X, y, "/workspace/data/EEG/models/")

Training for subject {0}: 
Partition 0: X_train = (2688, 257, 8), X_valid = (672, 257, 8), X_test = (840, 257, 8)
Model: "CNN1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 257, 8)]          0         
                                                                 
 conv1d_16 (Conv1D)          (None, 257, 10)           90        
                                                                 
 activation_19 (Activation)  (None, 257, 10)           0         
                                                                 
 conv1d_17 (Conv1D)          (None, 257, 50)           6550      
                                                                 
 activation_20 (Activation)  (None, 257, 50)           0         
                                                                 
 flatten (Flatten)           (None, 12850)             0         
              

2022-03-17 02:31:39.211189: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101
2022-03-17 02:31:39.665441: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


11/11 [==============================] - 2s 22ms/step - loss: 0.1691 - val_loss: 0.1740
Epoch 2/200
11/11 [==============================] - 0s 7ms/step - loss: 0.1488 - val_loss: 0.1705
Epoch 3/200
11/11 [==============================] - 0s 8ms/step - loss: 0.1474 - val_loss: 0.1697
Epoch 4/200
11/11 [==============================] - 0s 7ms/step - loss: 0.1470 - val_loss: 0.1693
Epoch 5/200
11/11 [==============================] - 0s 8ms/step - loss: 0.1466 - val_loss: 0.1692
Epoch 6/200
11/11 [==============================] - 0s 8ms/step - loss: 0.1460 - val_loss: 0.1685
Epoch 7/200
11/11 [==============================] - 0s 7ms/step - loss: 0.1455 - val_loss: 0.1683

KeyboardInterrupt: 

In [19]:
np.mean(aucs, axis=0)

0.8105828571428572

In [20]:
np.std(aucs, axis=0)

0.02280258892774197

In [21]:
X.shape















(4200, 257, 8)

In [8]:
import numpy as np

CE = np.load('/workspace/data/EEG/data/epochs/A-epo.npy')
lab = np.load("/workspace/data/EEG/data/epochs/A-labels.npy")

In [3]:
CE.shape

(8, 4200, 8, 257)

In [30]:
#!/usr/bin/env python3
# -*- coding: utf-8
#
# Montserrat Alvarado <amontserrat@gmail.com> / Gibran Fuentes-Pineda <gibranfp@unam.mx>
# DMAS-UAM / IIMAS-UNAM
# 2019
#
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.utils import get_custom_objects

def cecotti_normal(shape, dtype = None, partition_info = None):
    '''
    Initializer proposed by Cecotti et al. 2011:
    https://ieeexplore.ieee.org/document/5492691
    '''
    if len(shape) == 1:
        fan_in = shape[0]
    elif len(shape) == 2:
        fan_in = shape[0]
    else:
        receptive_field_size = 1
        for dim in shape[:-2]:
            receptive_field_size *= dim
        fan_in = shape[-2] * receptive_field_size

    return K.random_normal(shape, mean = 0.0, stddev = (1.0 / fan_in))

def scaled_tanh(z):
    '''
    Scaled hyperbolic tangent activation function, as proposed
    by Lecun 1989:
    http://yann.lecun.com/exdb/publis/pdf/lecun-89.pdf

    See also Lecun et al. 1998:
    http://yann.lecun.com/exdb/publis/pdf/lecun-98b.pdf
    '''
    return 1.7159 * K.tanh((2.0 / 3.0) * z)

get_custom_objects().update({'scaled_tanh': Activation(scaled_tanh)})

def CNN1(Chans = 6, Samples = 206):
    eeg_input    = Input(shape = (Samples, Chans))

    block1       = Conv1D(10, 1, padding = 'same',
                          data_format = 'channels_last',
                          bias_initializer = cecotti_normal,
                          kernel_initializer = cecotti_normal,
                          use_bias = True)(eeg_input)
    block1       = Activation('scaled_tanh')(block1)

    block1       = Conv1D(50, 13, padding = 'same',
                          data_format = 'channels_last',
                          bias_initializer = cecotti_normal,
                          kernel_initializer = cecotti_normal,
                          use_bias = True)(block1)
    block1       = Activation('scaled_tanh')(block1)

    flatten      = Flatten(name = 'flatten')(block1)
    dense        = Dense(100, activation = 'sigmoid')(flatten)
    prediction   = Dense(2, activation = 'sigmoid')(dense)

    return Model(inputs = eeg_input, outputs = prediction, name = 'CNN1')

In [31]:
#!/usr/bin/env python3
# -*- coding: utf-8
#
# Gibran Fuentes-Pineda <gibranfp@unam.mx>
# IIMAS, UNAM
# 2018
#
"""
Script to evaluate the CNN1 architecture (Lawhern et al., 2018) for single-trial subject-dependent P300 detection
"""
import argparse
import sys
import numpy as np
#from CNN1 import CNN1
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
#from tensorflow import set_random_seed
from sklearn.model_selection import *
from utils import *
import tensorflow.keras.backend as K

def evaluate_subject_models(data, labels, modelpath, subject):
    """
    Trains and evaluates CNN1 for each subject in the P300 Speller database
    using repeated stratified K-fold cross validation.
    """
    n_sub = data.shape[0]
    n_ex_sub = data.shape[1]
    n_samples = data.shape[2]
    n_channels = data.shape[3]
    aucs = np.zeros(5 * 10)
    print("Training for subject {0}: ".format(subject))
    cv = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 10, random_state = 123)
    for k, (t, v) in enumerate(cv.split(data[subject], labels[subject])):
        X_train, y_train, X_test, y_test = data[subject, t, :, :], labels[subject, t], data[subject, v, :, :], labels[subject, v]
        X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.2, shuffle = True, random_state = 456)
        print('Partition {0}: X_train = {1}, X_valid = {2}, X_test = {3}'.format(k, X_train.shape, X_valid.shape, X_test.shape))

        # channel-wise feature standarization
        sc = EEGChannelScaler(n_channels = n_channels)
        X_train = sc.fit_transform(X_train)
        X_valid = sc.transform(X_valid)
        X_test = sc.transform(X_test)

        model = CNN1(Chans = n_channels, Samples = n_samples)
        print(model.summary())
        model.compile(optimizer = 'adam', loss = 'mean_squared_error')

        # Early stopping setting also follows EEGNet (Lawhern et al., 2018)
        es = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 50, restore_best_weights = True)
        history = model.fit(X_train,
                            to_categorical(y_train),
                            batch_size = 256,
                            epochs = 200,
                            validation_data = (X_valid, to_categorical(y_valid)),
                            callbacks = [es])

        proba_test = model.predict(X_test)
        aucs[k] = roc_auc_score(y_test, proba_test[:, 1])
        print('S{0}, P{1} -- AUC: {2}'.format(subject, k, aucs[k]))
        K.clear_session()

    np.savetxt(modelpath + '/s' + str(subject) + '_aucs.npy', aucs)
    return aucs


In [32]:
auc_sub = evaluate_subject_models(mm, l, "/workspace/data/EEG/models/", 0)


Training for subject 0: 
Partition 0: X_train = (2688, 257, 8), X_valid = (672, 257, 8), X_test = (840, 257, 8)
Model: "CNN1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 257, 8)]          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 257, 10)           90        
                                                                 
 activation_4 (Activation)   (None, 257, 10)           0         
                                                                 
 conv1d_3 (Conv1D)           (None, 257, 50)           6550      
                                                                 
 activation_5 (Activation)   (None, 257, 50)           0         
                                                                 
 flatten (Flatten)           (None, 12850)             0         
                

2022-03-17 03:12:37.233842: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101
2022-03-17 03:12:37.697330: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


11/11 [==============================] - 2s 23ms/step - loss: 0.1915 - val_loss: 0.1650
Epoch 2/200
11/11 [==============================] - 0s 7ms/step - loss: 0.1437 - val_loss: 0.1612
Epoch 3/200
11/11 [==============================] - 0s 8ms/step - loss: 0.1401 - val_loss: 0.1563
Epoch 4/200
11/11 [==============================] - 0s 7ms/step - loss: 0.1346 - val_loss: 0.1473
Epoch 5/200
11/11 [==============================] - 0s 7ms/step - loss: 0.1277 - val_loss: 0.1407
Epoch 6/200
11/11 [==============================] - 0s 7ms/step - loss: 0.1198 - val_loss: 0.1342
Epoch 7/200
11/11 [==============================] - 0s 7ms/step - loss: 0.1151 - val_loss: 0.1290

In [53]:
CE.shape


(8, 4200, 8, 257)

In [4]:
mm = CE.reshape(CE.shape[0], CE.shape[1], CE.shape[3], CE.shape[2])

In [11]:
mm.shape

(8, 4200, 257, 8)

In [17]:
mm[0,0,:,0] == CE[0,0,0,:]

NameError: name 'mm' is not defined

In [6]:
b = np.transpose(CE,(0,1,3,2))

In [65]:
b[0,0,:,0] == CE[0,0,0,:]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [23]:
CE.shape[3]



257

In [11]:

lab[0,].shape

(4200,)

In [10]:
from sklearn.model_selection import *
CE=mm
subject=0
cv = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 10, random_state = 123)
for k, (t, v) in enumerate(cv.split(CE[subject], lab[subject])):
    X_train, y_train, X_test, y_test = CE[subject, t, :, :], lab[subject, t], CE[subject, v, :, :], lab[subject, v]
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.2, shuffle = True, random_state = 456)
    print('Partition {0}: X_train = {1}, X_valid = {2}, X_test = {3}'.format(k, X_train.shape, X_valid.shape, X_test.shape))


Partition 0: X_train = (2688, 257, 8), X_valid = (672, 257, 8), X_test = (840, 257, 8)
Partition 1: X_train = (2688, 257, 8), X_valid = (672, 257, 8), X_test = (840, 257, 8)
Partition 2: X_train = (2688, 257, 8), X_valid = (672, 257, 8), X_test = (840, 257, 8)
Partition 3: X_train = (2688, 257, 8), X_valid = (672, 257, 8), X_test = (840, 257, 8)
Partition 4: X_train = (2688, 257, 8), X_valid = (672, 257, 8), X_test = (840, 257, 8)
Partition 5: X_train = (2688, 257, 8), X_valid = (672, 257, 8), X_test = (840, 257, 8)
Partition 6: X_train = (2688, 257, 8), X_valid = (672, 257, 8), X_test = (840, 257, 8)
Partition 7: X_train = (2688, 257, 8), X_valid = (672, 257, 8), X_test = (840, 257, 8)
Partition 8: X_train = (2688, 257, 8), X_valid = (672, 257, 8), X_test = (840, 257, 8)
Partition 9: X_train = (2688, 257, 8), X_valid = (672, 257, 8), X_test = (840, 257, 8)
Partition 10: X_train = (2688, 257, 8), X_valid = (672, 257, 8), X_test = (840, 257, 8)
Partition 11: X_train = (2688, 257, 8), X_

In [16]:
CE[0].shape

(4200, 8, 257)

In [17]:

n_channels = CE.shape[3]
n_samples = CE.shape[2]

In [13]:
from utils import *

sc = EEGChannelScaler(n_channels = n_channels)
X_train = sc.fit_transform(X_train)
X_valid = sc.transform(X_valid)
X_test = sc.transform(X_test)

In [18]:
model = CNN1(Chans = n_channels, Samples = n_samples)
print(model.summary())
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

        # Early stopping setting also follows EEGNet (Lawhern et al., 2018)
es = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 50, restore_best_weights = True)

Model: "CNN1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 257, 8)]          0         
                                                                 
 conv1d (Conv1D)             (None, 257, 10)           90        
                                                                 
 activation_1 (Activation)   (None, 257, 10)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 257, 50)           6550      
                                                                 
 activation_2 (Activation)   (None, 257, 50)           0         
                                                                 
 flatten (Flatten)           (None, 12850)             0         
                                                                 
 dense (Dense)               (None, 100)               1285100

2022-03-17 03:09:22.010067: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-17 03:09:22.725967: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30988 MB memory:  -> device: 0, name: Tesla V100-SXM3-32GB, pci bus id: 0000:bc:00.0, compute capability: 7.0


NameError: name 'EarlyStopping' is not defined

In [19]:
y_train

array([1, 1, 1, ..., 1, 2, 1])

In [21]:
from tensorflow.keras.utils import to_categorical

to_categorical(y_train)

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]], dtype=float32)

In [29]:
l = lab-1

In [26]:
lab.shape

(8, 4200)

In [28]:
unique, counts = np.unique(lab, return_counts=True)
dict(zip(unique, counts))

{1: 28000, 2: 5600}